# Gettings started with Amazon `Bedrock`

![](images/bedrock_models.png)

### Create a `bedrock` client

In [1]:
import json
import boto3
import base64



#Create the connection to Bedrock
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-west-2', 
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2', 
    
)

### List of all available models

In [2]:
# List all foundation models
all_llms = [ model['modelId'] for model in bedrock.list_foundation_models()['modelSummaries']]
all_llms

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'stability.stable-diffusion-xl',
 'stability.stable-diffusion-xl-v0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'ai21.j2-grande-instruct',
 'ai21.j2-jumbo-instruct',
 'ai21.j2-mid',
 'ai21.j2-mid-v1',
 'ai21.j2-ultra',
 'ai21.j2-ultra-v1',
 'anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0',
 'cohere.command-text-v14:7:4k',


### Define prompt and model parameters 

### With `Amazon Titan`

In [3]:
prompt_data = """Write me a poem about apples"""

text_gen_config = {
                    "maxTokenCount": 512,
                    "stopSequences": [], 
                    "temperature": 0,
                    "topP": 0.9
                }

body = json.dumps({
                        "inputText": prompt_data,
                        "textGenerationConfig": text_gen_config  
                    })

model_id = 'amazon.titan-tg1-large'
accept = 'application/json' 
content_type = 'application/json'

#### Invoke model 


In [4]:
response = bedrock_runtime.invoke_model(
                                            body=body, 
                                            modelId=model_id, 
                                            accept=accept, 
                                            contentType=content_type
                                        )

#### Print response

In [5]:
response_body = json.loads(response['body'].read())
print(response_body['results'][0]['outputText'])


Here is a poem about apples:

Apples are so round and bright,
They make my mouth water with delight.
Their skin is so smooth and crisp,
I can't help but eat them every day.

They come in red, yellow, and green,
But my favorite is the pink and serene.
I eat them with peanut butter,
They're the perfect snack for any time.


### With `Anthropic (Claude v3 (Sonnet))`

In [6]:
prompt_data = """What Large in LLM means"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

#### Invoke model 


In [7]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

#### Print response

In [8]:
response_body = json.loads(response.get("body").read())
for output in response_body.get("content", []):
    print(output["text"])

In the context of LLMs (Large Language Models), the term "Large" refers to the massive size and complexity of these models in terms of the number of parameters they contain.

LLMs are deep learning models trained on vast amounts of text data to acquire broad knowledge and capabilities in natural language processing tasks. They have achieved remarkable performance by scaling up the model size, measured by the number of parameters (weights and biases) they possess.

Some key points about the "Large" in LLMs:

1. Parameter Count: LLMs typically have hundreds of billions or even trillions of parameters. For example, GPT-3 (one of the most well-known LLMs) has 175 billion parameters, while the more recent PaLM model has 540 billion parameters.

2. Model Capacity: The large number of parameters allows LLMs to capture and model intricate patterns and relationships in the training data, enabling them to handle complex natural language tasks with high accuracy and fluency.

3. Training Data: To

### With `Mistral (mixtral-8x7b)`

In [9]:
prompt_data = """<s>[INST]Craft a Python function to convert Celsius to Fahrenheit. If water boils at 100°C, what's that in Fahrenheit?[/INST]"""

body = json.dumps({ 
    'prompt': prompt_data,
    'max_tokens': 200,
    'top_p': 0.9,
    'temperature': 0.2,
})


modelId = 'mistral.mixtral-8x7b-instruct-v0:1'
accept = 'application/json'
contentType = 'application/json'

#### Invoke model 


In [10]:
response = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                        modelId=modelId, 
                                        accept=accept, 
                                        contentType=contentType)

#### Print response

In [11]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('outputs')[0].get('text'))

 Here is a simple Python function to convert Celsius to Fahrenheit:

```python
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32
```

Now, let's use this function to convert 100 degrees Celsius to Fahrenheit:

```python
print(celsius_to_fahrenheit(100))
```

When you run this code, it will output `212.0`, which is the boiling point of water in Fahrenheit.


### Syntax Multi-Modal models from `Anthropic Claude v3 Models`

In [12]:
with open("images/cat.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')

payload = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": "Write me a detailed description of this photo."
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body_bytes,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)


In [13]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('content')[0].get('text'))

The image depicts a cute, cartoon-style illustration of a tabby cat. The cat has large, expressive eyes with brown irises that give it an adorable and endearing look. Its face features a small pink nose, white whiskers, and a smiling expression, revealing its front teeth.

The cat's body is orange and white in color, with distinct striped markings reminiscent of a tabby cat breed. Its front paws are white, while the rest of its legs and tail are striped orange and white. The tail curves elegantly behind the cat's body.

The illustration has a simplistic yet charming style, with a gray background that allows the vibrant colors of the cat to stand out. The overall image conveys a sense of playfulness and warmth, capturing the endearing nature of a beloved feline companion.
